
# Denoising Diffusion Implicit Model
In diesem Notebook soll es darum gehen, die Grundlagen der Technologie von den zurzeit viel bepriesenen Image Generator Modellen wie StableDiffusion zu verstehen.

Jeder Code Block kann durch klicken des "Play" Buttons oben Links ausgeführt werden. Es ist wichtig die Blöcke in der richtigen Reihenfolge auszuführen, ansonsten kann es zu Fehlern kommen.




##Import und Setup

Jedes Programm startet mit sogenannten Imports. Hierbei handelt es sich um das Laden von anderen, bereits programmierten Scripts.
In unserem Fall benutzen wir TensorFlow, ein Framework von Google, welches dazu dient Machine Learning Modelle zu programmieren.



In [ ]:
import tensorflow as tf #Dieses Package definiert die eigentlichen neuronalen Netzstrukturen, sowie Datentypen und Rechenoperationen um sie zu manipulieren
import tensorflow_datasets as tfds #Dieses Package dient dem Download von Datensets, damit wir sie nicht per Hand downloaden müssen
import matplotlib.pyplot as plt #Ein Package, welches dem plotten von Daten dient. Wir benutzen es um die Bilder aus dem Datenset zu plotten, sowie unsere Trainingsergebnisse
import keras #Ein High Level Framework welches auf TensorFlow aufbaut. Hat viele nützliche Funktionen die uns das Leben erleichtern

Außerdem brauchen wir 2 scripts, die das eigentliche DDIM implementieren. Dessen verständnis ist nicht notwendig, falls du neugierig bist und etwas python verstehst, kannst du aber gerne den Source code anschauen. Klappe einfach die folgende section aus, klicke auf "play" und klappe sie wieder zu.

### Diffusion Model

In [ ]:
import tensorflow as tf
import keras
from keras import layers
import math
import matplotlib.pyplot as plt


embedding_dims = 32
embedding_max_frequency = 1000.0
embedding_min_frequency = 1.0


def sinusoidal_embedding(x):
    frequencies = tf.exp(
        tf.linspace(
            tf.math.log(embedding_min_frequency),
            tf.math.log(embedding_max_frequency),
            embedding_dims // 2,
        )
    )
    angular_speeds = 2.0 * math.pi * frequencies
    embeddings = tf.concat(
        [tf.sin(angular_speeds * x), tf.cos(angular_speeds * x)], axis=3
    )
    return embeddings


def ResidualBlock(width):
    def apply(x):
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = layers.Conv2D(width, kernel_size=1)(x)
        x = layers.BatchNormalization(center=False, scale=False)(x)
        x = layers.Conv2D(
            width, kernel_size=3, padding="same", activation=keras.activations.swish
        )(x)
        x = layers.Conv2D(width, kernel_size=3, padding="same")(x)
        x = layers.Add()([x, residual])
        return x

    return apply


def DownBlock(width, block_depth):
    def apply(x):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(width)(x)
            skips.append(x)
        x = layers.AveragePooling2D(pool_size=2)(x)
        return x

    return apply


def UpBlock(width, block_depth):
    def apply(x):
        x, skips = x
        x = layers.UpSampling2D(size=2, interpolation="bilinear")(x)
        for _ in range(block_depth):
            x = layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x)
        return x

    return apply


def get_network(image_size, widths, block_depth):
    noisy_images = keras.Input(shape=(image_size, image_size, 3))
    noise_variances = keras.Input(shape=(1, 1, 1))

    e = layers.Lambda(sinusoidal_embedding)(noise_variances)
    e = layers.UpSampling2D(size=image_size, interpolation="nearest")(e)

    x = layers.Conv2D(widths[0], kernel_size=1)(noisy_images)
    x = layers.Concatenate()([x, e])

    skips = []
    for width in widths[:-1]:
        x = DownBlock(width, block_depth)([x, skips])

    for _ in range(block_depth):
        x = ResidualBlock(widths[-1])(x)

    for width in reversed(widths[:-1]):
        x = UpBlock(width, block_depth)([x, skips])

    x = layers.Conv2D(3, kernel_size=1, kernel_initializer="zeros")(x)

    return keras.Model([noisy_images, noise_variances], x, name="residual_unet")




max_signal_rate = 0.95
min_signal_rate = 0.02
ema = 0.999
plot_diffusion_steps = 50


class DiffusionModel(keras.Model):
    def __init__(self, image_size, widths, block_depth, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.normalizer = keras.layers.Normalization()
        self.image_size = image_size
        self.network = get_network(image_size, widths, block_depth)
        self.ema_network = keras.models.clone_model(self.network)

    def compile(self, **kwargs):
        super().compile(**kwargs)
        self.noise_loss_tracker = keras.metrics.Mean(name="noise_loss")
        self.image_loss_tracker = keras.metrics.Mean(name="image_loss")

    @property
    def metrics(self):
        return[self.noise_loss_tracker, self.image_loss_tracker]

    def denormalize(self, images):
        images = self.normalizer.mean + images * self.normalizer.variance**0.5
        return tf.clip_by_value(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        # diffusion times -> angles
        start_angle = tf.acos(max_signal_rate)
        end_angle = tf.acos(min_signal_rate)

        diffusion_angles = start_angle + \
            diffusion_times * (end_angle - start_angle)

        # angles -> signal and noise rates
        signal_rates = tf.cos(diffusion_angles)
        noise_rates = tf.sin(diffusion_angles)
        # note that their squared sum is always: sin^2(x) + cos^2(x) = 1

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates, training):
        # the exponential moving average weights are used at evaluation
        if training:
            network = self.network
        else:
            network = self.network

        # predict noise component and calculate the image component using it
        pred_noises = network(
            [noisy_images, noise_rates**2], training=training)

        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates

        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps):
        # reverse diffusion = sampling
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps

        # important line:
        # at the first sampling step, the "noisy image" is pure noise
        # but its signal rate is assumed to be nonzero (min_signal_rate)
        next_noisy_images = initial_noise
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images

            # separate the current noisy image to its components
            diffusion_times = tf.ones((num_images, 1, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(
                diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=False
            )
            # network used in eval mode

            # remix the predicted components using the next signal and noise rates
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )
            # this new noisy image will be used in the next step

        return pred_images

    def generate(self, num_images, diffusion_steps):
        if self.image_size >= 256:
            # Too big, need to generate one by one.
            images = []
            for i in range(num_images):
                initial_noise = tf.random.normal(
                    shape=(1, self.image_size, self.image_size, 3))
                generated_images = self.reverse_diffusion(
                    initial_noise, diffusion_steps)
                generated_images = self.denormalize(generated_images)
                images.append(tf.squeeze(generated_images))
            return images

        else:
            # noise -> images -> denormalized images
            initial_noise = tf.random.normal(
                shape=(num_images, self.image_size, self.image_size, 3))
            generated_images = self.reverse_diffusion(
                initial_noise, diffusion_steps)
            generated_images = self.denormalize(generated_images)
            return generated_images

    def train_step(self, images):
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=True)
        noises = tf.random.normal(
            shape=(self.batch_size, self.image_size, self.image_size, 3))

        # sample uniform random diffusion times
        diffusion_times = tf.random.uniform(
            shape=(self.batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly

        noisy_images = signal_rates * images + noise_rates * noises

        with tf.GradientTape() as tape:
            # train the network to separate noisy images to their components
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=True
            )

            noise_loss = self.loss(noises, pred_noises)  # used for training
            image_loss = self.loss(images, pred_images)  # only used as metric

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(
            zip(gradients, self.network.trainable_weights))

        self.noise_loss_tracker.update_state(noise_loss)
        self.image_loss_tracker.update_state(image_loss)

        # track the exponential moving averages of weights
        for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # KID is not measured during the training phase for computational efficiency
        return {m.name: m.result() for m in self.metrics[:-1]}

    def plot_images(self, epoch=None, logs=None, num_rows=3, num_cols=6):
        # plot random generated images for visual evaluation of generation quality
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=plot_diffusion_steps,
        )
        # print(generated_images)

        plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
        for row in range(num_rows):

            #image = generated_images[row].numpy()
            #plt.subplot(1, 5, row + 1)
            #img = Image.fromarray(image, 'RGB')
            # plt.imshow(image)
            for col in range(num_cols):
                index = row * num_cols + col
                plt.subplot(num_rows, num_cols, index + 1)
                image = generated_images[index].numpy()
                plt.imshow(image)
                plt.axis("off")
                # plt.show()
        plt.tight_layout()
        plt.show()
        plt.savefig("epoch_" + str(epoch))
        plt.close()



### Datenset

Als nächstes müssen wir unsere trainingsdaten Laden. In diesem Fall benutzen wir das Image set "Oxford flowers 102", ein Datenset mit etwa 6000 Bildern von Blüten.

Im nächsten code Block wird das Datenset heruntergeladen (was eine weile dauern kann), und ein paar Beispielbilder ausgegeben:

In [ ]:
dataset = tfds.load("oxford_flowers102", split="train[:80%]+validation[:80%]+test[:80%]", shuffle_files=True)

plt.rcParams["figure.figsize"] = [30, 15]
plt.rcParams["figure.autolayout"] = True
ctr = 0
for data in dataset:
  image = data["image"]
  image = image.numpy()
  plt.subplot(1, 5, ctr+1)
  plt.title('Label {}'.format(data["label"]))
  plt.imshow(image, cmap=plt.cm.binary)
  ctr += 1
  if ctr == 5:
    break

Die Nummer die neben dem Label ausgegeben wird entspricht der "Klasse" der dieses Bild angehört. In diesem Fall steht die Nummer für eine bestimmte Pflanzenart. Diese Labels sind wichtig wenn man z.B. ein neuronales Netz bauen möchte, was die Pflanzenart anhand eines Bildes der Blüte erkennt. Dann kann man sie benutzen um dem Netz feedback zu geben ob es richtig oder falsch liegt.

Außerdem sind sie wichtig wenn man bestimmte Blüten generieren möchte. Beispielsweise wäre es cool dem Netz sagen zu können man möchte gerne ein Bild einer Sonnenblume haben. Dies ist mit DDIM Models möglich, aber nicht Teil dieses Notebooks. Entwickelt man diese Technik dann weiter, kann man sogar Text-to-Image generation machen, wie z.B. StableDiffusion von OpenAI.





####<font color="blue">Frage: Was Fällt auf bei den Bildern auf was für unser Netz relevant sein könnte?</font>


<font color="blue">*Antwort: Die Bilder haben verschiedene Größen. Das ist ein Problem für neuronale Netzwerke. Denn diese Erwarten einen Input von immer gleicher größe. Schließlich lernen sie exakte operationen auf dem Input durchzuführen. Deshalb müssen wir bevor wir loslegen können die Bilder alle auf die selbe Größe bringen.*</font>

## Vorverarbeitung der Daten

Dies ist ein wichtiger Teil in Machine Learning. Selbst wenn das neuronale Netz einwandfrei funktioniert, kann es sein dass das Training fehl schlägt, wenn es Probleme mit dem Datenset gibt. Dies beinhaltet das sogenannte normalisieren von Daten, aber auch eine Analyse des Datensets selbst. Gibt es vielleicht einen möglichen bias?

Beispiel hier wäre ein oft genutztes Datenset: CelebA
Ein Datenset mit Gesichtern von Berühmtheiten. Es wird oft benutzt um facial recognition oder generation zu trainieren. Allerdings hat es einen klaren bias:

####<font color="blue">Frage: Welchen bias erwartest du in einem Datenset von Gesichtern von berühmten Personen?</font>

<font color="blue">*Berühmtheiten haben oft ein sehr symmetrisches Gesicht und sind überwiegend weiß. Dies hat in der Vergangenheit dazu geführt, dass z.B. Facial Login methoden auf Smartphones bei nicht-weißen Personen wesentlich schlechter funktioniert haben.*</font>

####<font color="blue">Frage: Welchen bias erwartest du in unserem Datenset?</font>

<font color="blue">*Antwort: Nicht jede Pflanzenart ist hier gleich verteten. Viele sogar gar nicht. Außerdem Blühen Pflanzen zu verschiedenen Tages/ Jahreszeiten, was für verschiedene Lichtverhältnisse beim aufnehmen der Photos sorgt.*</font>

###Normalisieren der Bilder

In unserem Fall geben wir uns mit dem Normalisieren der Bilder zufrieden. Das bedeutet hier, das alle Bilder auf die selben Maße zugeschnitten werden. Wir müssen hier leider auf 64x64 zurückgreifen. Dies ist der Performance geschuldet, denn wir müssen die Trainingszeit stark optimieren um im Rahmen dieses Workshops überhaupt Ergebnisse zu erhalten. Falls du zuhause nochmal rumspielen möchtest, versuche doch mal im nachfolgenden code block die IMG_SIZE Variable auf 128 oder 256 zu setzen.

Außerdem werden die Farbkanäle der Bilder auf 1 normiert. Normalerweise werden Farben im RGB-Format (Rot-Gelb-Grün) kodiert, auf einer Skala von 1-255. Ein RGB Wert von 0, 0, 0 bedeutet z.B. weder Rot, noch Gelb, noch Grün ist vorhanden, und die Farbe ist demnach schwarz.
Neuronale Netzwerke lieben input der zwischen 0 und 1 liegt. deshalb teilen wir jeden RGB wert durch 255. damit ist z.B. 255, 255, 255 auf 1, 1, 1 geschrumpft. Die Information ist die selbe, aber es erleichtert das Training.

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 128

def format_image(data):
    height = tf.shape(data["image"])[0]
    width = tf.shape(data["image"])[1]
    crop_size = tf.minimum(height, width)
    image = tf.image.crop_to_bounding_box(
        data["image"],
        (height - crop_size) // 2,
        (width - crop_size) // 2,
        crop_size,
        crop_size,
    )
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE], antialias=True)
    return tf.clip_by_value(image / 255.0, 0.0, 1.0)


training_set = (dataset
                   .map(format_image, num_parallel_calls=tf.data.AUTOTUNE)
                   .cache()
                   .repeat(5)
                   .shuffle(10 * BATCH_SIZE)
                   .batch(BATCH_SIZE, drop_remainder=True)
                   .prefetch(buffer_size=tf.data.AUTOTUNE)
)

##Definition des Models & Training

Nun geht es endlich an das eigentliche Model. Die eigentliche implementierung führt hier ein wenig weit. Bei interesse kannst du gerne einen Blick in die scripts "diffusion_model.py", in der der Trainingsprozess definiert ist, sowie "model.py", in dem das eigentliche neuronale netz gebaut wird werfen.

Für unsere Zwecke reicht ein high-level Überblick über das Netzwerk.
Wie in der Einführung erklärt, beruht ein Diffusion Model auf einem UNet.
Das bedeutet der Input besitzt die selben dimensionen (in unserem Fall 64x64) wie der output. Im innern des Netzwerks werden dann die dimensionen des Inputs Schritt für Schritt erniedrigt, und dann wieder erhöht. Dies führt dazu, dass das Model lernen muss, wesentliche Informationen in kondensierter Form darzustellen. Ein Mensch merkt sich ein Bild auch nicht Pixel für Pixel. Vielmehr benutzen Menschen abstrakte Aussagen, wie z.B. "Das Bild zeigt eine schwebende, rote Rose über dem offenen Meer". Neuronale Netze verarbeiten Informationen zwar ganz anders als wir, dennoch müssen auch sie Lernen, welche Informationen wichtiger sind als andere.

Zur Errinnerung, hier das stark vereinfachte Diagram, was das innere (UNet) unseres neuronalen Netzwerks beschreibt:

![image](https://drive.google.com/uc?export=view&id=1AiocI2fC6rxr3XusXTOBjA3IrE0XxzJU)

#### <font color="blue">Frage: Welche Abfolge von dimensionen hälst du im Inneren unseres UNets für angemessen?</font>

<font color="blue">*Die input dimensionen sind 64x64. Wir wollen diese dimensionen schrumpfen lassen und dann wieder bei 64 herauskommen. Deshalb bietet sich eine Abfolge von 64 -> 32 -> 24 -> 16 -> 24 -> 32 -> 64 an.*
*Die kleinste Stelle im UNet muss immernoch hinreichend groß sein, um alle wichtigen Informationen des Input Bilds zu erfassen. Wir können beispielsweise nicht auf ein 1x1 Bild heruntergehen, denn ein einziger Pixel ist selbstverständlich zu wenig um den Inhalt eines Bilds zu beschreiben. Wie tief man gehen kann ist ein Erfahrungswert, der durch trial and error erlangt wird.*

*Diese Zahlen müssen nicht zwangsläufig Vielfache von 2 sein. Theoretisch könnte man auch 64 -> 30 -> 20 -> 10 -> 20 ->30 -> 64 benutzen. Allerdings haben Vielfache (insbesondere Potenzen) von 2 zahlreiche Vorteile in Datenverarbeitungen.*
</font>

#### Das Training:
Das ausführen des folgenden codeblocks definiert das Model, und startet das Training. Dies kann mehrere Minuten dauern. Wir haben versucht das Training soweit wie möglich zu optimieren. Das führt allerdings zu suboptimalen Ergebnissen. Wenn du die Grenzen des Models austesten möchtest, dann setze oben die IMG_SIZE auf 256, die UNET_SHAPE auf [96, 128, 192] und EPOCHS auf mindestens 30

Während das Training läuft werden nach jeder Epoche ein paar Beispielbilder angezeigt. Diese zeigen, was das Model zu der jeweiligen Epoche kann. Du solltest beobachten können, wie die Bilder von Epoche zu Epoche besser werden.

Damit du nicht zu lange wartest, solltest du ruhig schonmal zu dem Abschnitt "Ergebnisse" vorspringen. Das gesamte Training sollte ca. 5-10 minuten dauern.

In [ ]:
UNET_SHAPE = [16, 24, 32]
BLOCK_DEPTH = 2
EPOCHS = 10

model = DiffusionModel(IMG_SIZE, UNET_SHAPE, BLOCK_DEPTH, BATCH_SIZE)
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=1e-3
), loss=keras.losses.mean_absolute_error )

model.fit(training_set, epochs=EPOCHS, callbacks=[keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images)])

##Ergebnisse

Während das Model trainiert wollen wir hier ein paar Interessante Ergebnisse ansprechen. im oberen Beispiel wird für 10 Epochen trainiert. Zur Errinnerung: Eine Epoche bedeutet, dass alle Bilder im Trainingsdatenset einmal trainiert wurden. Im regelfall werden Modelle für deutlich mehr als 10 Epochen trainiert. Hier sind deshalb ein paar Beispiele nach 50 Epochen des oben definierten Modells:

![picture](https://drive.google.com/uc?export=view&id=1-QmnJaBKi-ODG9Imij9TutGQQVxdO9OG)

Lass uns das damit Vergleichen, was tatsächlich als trainingsinput verwendet wurde. Denn wir errinnern uns: Wir haben die Daten vorverarbeitet (Deutlich runterskaliert auf 64x64)

In [ ]:
sample_set = dataset.map(format_image, num_parallel_calls=tf.data.AUTOTUNE)
sample_images = sample_set.take(50)
plt.rcParams["figure.figsize"] = [30, 15]
plt.rcParams["figure.autolayout"] = True
ctr = 0
for row in range(3):
  for col in range(6):
    index = row * 6 + col
    image = list(sample_images.as_numpy_iterator())[index]
    #image = data["image"]
    #image = image.numpy()
    plt.subplot(3, 6, index + 1)
    plt.axis("off")
    plt.imshow(image, cmap=plt.cm.binary)

Das Ergebnis unseres Trainings ist hiervon weit Entfernt.

#### <font color="blue">Frage: Wie würdest du die Unterschiede beschreiben?</font>

<font color="blue">*Die echten Bilder haben wesentlich mehr Details. Das Netz hat lediglich gelernt, dass "Blüten" ein farbiger Blob auf einem grünen Hintergrund sind.*</font>

#### <font color="blue">Frage: Wie könntest du dir die Unterschiede Erklären?</font>

<font color="blue">*Antwort: Ein Mangel an Details in generativen Netzwerken deutet oft auf fehlende Mächtigkeit des Models hin. Dem Model gelingt es grob die Struktur (farbiger Blob auf grünem Hintegrund) zu Lernen. Es hat auch verstanden, dass die meisten Blüten sich aus wenigen Farben zusammensetzen. Es hat aber nicht genug "Power" um Beispielsweise verschiedene Formen von Blütenblättern zu verstehen.*
</font>

### Vortrainierte Version
Zu wenig Power? Na dann, machen wir unser Netzwerk größer!

Ich habe im Vorfeld des Workshops eine Version des Netzwerks auf unserem AG Server trainiert. Dieses nimmt 64x64 Bilder, und hat deutlich mehr Parameter (Unet Shape 64 -> 128 -> 96 -> 64 -> 32 -> 64 -> 96 -> 128 -> 64)
Nach 50 Epochen haben wir folgende Ergebnisse:

![image](https://drive.google.com/uc?export=view&id=1-on2DtyTxzOwJPkGI0qQYLUvI45PnVz_)

Das sieht doch schon viel mehr nach Blüten aus!
Es lassen sich einzelne Blütenblätter erkennen, der Hintergrund ist detaillierter, und die Blüten haben realistische Farbverläufe.
Einige Blüten haben aber dennoch merkwürdige, nicht ganz naturgemäße Formen :-)

Lass uns nun einen genaueren Blick auf die Generation dieser Bilder werfen. Wie in der Einführung besprochen, ist dieses Model ein iteratives Model. Ein Bild (Am Anfang buntes rauschen) wird in das Model gefüttert. Dieses versucht ein klein wenig rauschen zu entfernen, und gibt ein neues Bild aus. Dieses Bild wird dann wieder in das Model gefüttert. Hier eine visualisierung des Prozesses als .gif animiert, und als einzelne Schritte:

![image](https://drive.google.com/uc?export=view&id=1HMrcVuuJvbTTXRSChBPbvIjcs1f5nSeb)
![image](https://drive.google.com/uc?export=view&id=1-qaFvDfygCAXXTPv9SiFkVI_cw5PSmcY)

#### <font color="blue">Frage: Wie würdest du den Prozess beschreiben?</font>

<font color="blue">*Antwort: Das Netz scheint innerhalb der ersten durchläufe eine Farbe und grobe Form festzulegen. Später kommen dann Details dazu. Es fällt auf, dass die ersten Schritte deutlich mehr Änderungen vornehmen. Vergleiche hier den unterschied von Schritt 1 -> 2 im vergleich zu Schritt 25 -> 26*
</font>

### Quellen

Der hier verwendete code stammt von einem [Keras Guide](https://keras.io/examples/generative/ddim/) und wurde von mir zu Zwecken dieses Workshops angepasst. Alle Bilder wurden auf unserem AG Server generiert, Diagramme stammen von mir.